In [ ]:
!pip install pandas numpy scikit-learn tensorflow keras matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow import keras

In [ ]:
import pandas as pd

# Define column names as per NSL-KDD dataset
columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
           'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
           'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
           'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
           'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
           'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
           'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
           'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate',
           'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack', 'level']

# Load dataset
df = pd.read_csv("KDDTrain+.txt", names=columns)

# Display first five rows
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [ ]:
# Display dataset shape
print(f"Dataset shape: {df.shape}")

# Check data types and missing values
print(df.info())

# Show unique attack types
print(f"Unique attack types:\n{df['attack'].unique()}")

Dataset shape: (125973, 43)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125973 non-null  int64  
 1   protocol_type                125973 non-null  object 
 2   service                      125973 non-null  object 
 3   flag                         125973 non-null  object 
 4   src_bytes                    125973 non-null  int64  
 5   dst_bytes                    125973 non-null  int64  
 6   land                         125973 non-null  int64  
 7   wrong_fragment               125973 non-null  int64  
 8   urgent                       125973 non-null  int64  
 9   hot                          125973 non-null  int64  
 10  num_failed_logins            125973 non-null  int64  
 11  logged_in                    125973 non-null  int64  
 12  num_compromised              1

In [ ]:
# Define attack categories
attack_mapping = {
    'apache2': 'DoS', 'back': 'DoS', 'land': 'DoS', 'neptune': 'DoS', 'mailbomb': 'DoS',
    'pod': 'DoS', 'processtable': 'DoS', 'smurf': 'DoS', 'teardrop': 'DoS', 'udpstorm': 'DoS', 'worm': 'DoS',

    'ipsweep': 'Probe', 'mscan': 'Probe', 'nmap': 'Probe', 'portsweep': 'Probe', 'saint': 'Probe', 'satan': 'Probe',

    'buffer_overflow': 'Privilege Escalation', 'loadmodule': 'Privilege Escalation', 'perl': 'Privilege Escalation',
    'ps': 'Privilege Escalation', 'rootkit': 'Privilege Escalation', 'sqlattack': 'Privilege Escalation', 'xterm': 'Privilege Escalation',

    'ftp_write': 'Remote Access', 'guess_passwd': 'Remote Access', 'http_tunnel': 'Remote Access', 'imap': 'Remote Access',
    'multihop': 'Remote Access', 'named': 'Remote Access', 'phf': 'Remote Access', 'sendmail': 'Remote Access',
    'snmpgetattack': 'Remote Access', 'snmpguess': 'Remote Access', 'spy': 'Remote Access', 'warezclient': 'Remote Access',
    'warezmaster': 'Remote Access', 'xclock': 'Remote Access', 'xsnoop': 'Remote Access'
}

# Apply mapping
df['attack_category'] = df['attack'].map(attack_mapping).fillna('Normal')

# Display counts of each category
print(df['attack_category'].value_counts())

attack_category
Normal                  67343
DoS                     45927
Probe                   11656
Remote Access             995
Privilege Escalation       52
Name: count, dtype: int64


In [ ]:
from sklearn.preprocessing import LabelEncoder

# One-hot encode protocol_type, service, and flag
df = pd.get_dummies(df, columns=['protocol_type', 'service', 'flag'])

# Label encode attack_category
encoder = LabelEncoder()
df['attack_category_encoded'] = encoder.fit_transform(df['attack_category'])

# Drop old attack columns
df = df.drop(columns=['attack', 'attack_category'])

# Display first 5 rows
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,attack_category_encoded
0,0,491,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,True,False,1
1,0,146,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,True,False,1
2,0,0,0,0,0,0,0,0,0,0,...,False,False,False,True,False,False,False,False,False,0
3,0,232,8153,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,True,False,1
4,0,199,420,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,True,False,1


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Select numerical columns (excluding encoded attack_category)
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols.remove('attack_category_encoded')  # Exclude target

# Apply Min-Max Scaling
scaler = MinMaxScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Display first 5 rows
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,attack_category_encoded
0,0.0,3.558064e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,True,False,1
1,0.0,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,True,False,1
2,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,0
3,0.0,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,False,False,False,False,False,False,False,True,False,1
4,0.0,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,False,False,False,False,False,False,False,True,False,1


# Step 7: Splitting the Dataset
We'll now split the dataset into:

1) Training Set (80%) → Used to train the model
2) Testing Set (20%) → Used to evaluate the model's performance

In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df.drop(columns=['attack_category_encoded'])  # Features
y = df['attack_category_encoded']  # Target labels

# Split the dataset (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Display dataset shapes
print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

In [ ]:
Step 8: Model Selection & Training
Since we are building an AI-based Intrusion Detection System (IDS), we will experiment with different machine learning and deep learning models.

For a thorough approach, we’ll try:
1. Traditional ML models: Random Forest, SVM, etc.
2. Deep Learning models: Neural Networks (ANN), LSTMs, CNNs

## 8.1 Exploring Traditional ML models
## 8.1.1 Random Forest Classifier 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate performance
accuracy_rfc = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy: {accuracy_rfc:.4f}")

# Detailed classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
Observations:
Extremely high accuracy → Indicates strong classification performance
High precision & recall → No significant false positives or false negatives
Class Imbalance Noted → Some attack categories (e.g., Privilege Escalation) have fewer instances, affecting recall (0.90 for class 2).

## 8.1.2 Support Vector Machine

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Train the SVM model
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
svm_model.fit(X_train, y_train)

# Predictions
y_pred_svm = svm_model.predict(X_test)

# Evaluation
svm_accuracy = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred_svm))

## 8.1.3 KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Predictions
y_pred_knn = knn_model.predict(X_test)

# Evaluation
knn_accuracy = accuracy_score(y_test, y_pred_knn)
print(f"KNN Accuracy: {knn_accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred_knn))

## 8.1.4 XGBoost

In [ ]:
!pip install xgboost